Guys!

This notebook is to help but also to ask for help, because I made it work for the TRAINING images, but not for the PREDICTED image, which is the difficult part.

Those of you who want some help, consider this:
- I use a Neural Network of 4 layers to predict the values of each pixel of the output matrix. It works quite fine! It can be used to TRAIN ANY task but with the following constraint.
- Constraint: it only works if the SHAPE of the train images are the same. In other words, the dimensions of the input matrices must be the same. Also the dimensions of the output matrices. If a task fulfill this, it works. Example: Input images: all of them (3x3) -->  Output images: all of them (9x9). This works fine!
                                                        
- PROBLEM: I'm not able to make the NeuralNetwork to learn patterns. Instead, it learns the weights to reach any output. This is the reason why it works for the training. I get different weights for each image of the task.
- This is a problem of OVERFITTING, and is the main problem of why any of us are reaching good results. 
- This problem can be seen very well in the fact that I took the average of the weights for the training images to predict the test image, and of course, it's not working.

SO,
How to solve overfitting?
1. Imagine our input is an image of 10x10=100 neurons
2. In a traditional Neural Network we can not train 100 weights with 5 examples -->  **VERY HIGH VARIANCE (OVERFITTING)** h(x)=theta0 + theta1*x1 + ... + theta100 * x100. It's very possible that we get the expected result in the training but not in the test.
3. Possible solutions:  
   - increase m - augment the data
   - use regularization (increase lambda)
   - apply PCA(Principal Component Analysis) to data compression --> less features BUT it is not recommended to avoid Overfitting. 
   - using another activation function: ReLu ?
   
Guys! At this point, I need your help. I'm thinking to augment the data and to apply regularization. Do you also think is the RIGHT decision?

Thanks a lot for your time :)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import matplotlib.pyplot as plt
from matplotlib import colors


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from pathlib import Path

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

training_tasks = sorted(os.listdir(training_path))


In [ ]:
# ======================================================================================
# SHOW THE IMAGES OF 1 TASK
# ======================================================================================

cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)

def plot_one(ax, i,train_or_test,input_or_output):

    
    input_matrix = task[train_or_test][i][input_or_output]
    ax.imshow(input_matrix, cmap=cmap, norm=norm)
    ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
    ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
    ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(train_or_test + ' '+input_or_output)

def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    print('Nº of training examples: ', num_train)
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(axs[0,i],i,'train','input')
        plot_one(axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(axs[0],0,'test','input')
        plot_one(axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(axs[0,i],i,'test','input')
            plot_one(axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 
    
plt.figure(figsize=(5, 2), dpi=200)
plt.imshow([list(range(10))], cmap=cmap, norm=norm)
plt.xticks(list(range(10)))
plt.yticks([])
plt.show()

In [ ]:
# ======================================================================================
# SAVE ALL THE DATA IN dict_tasks and dict_tasks_shape
# ======================================================================================


dict_tasks = {}
dict_tasks_shape = {}
for i in range(0,len(training_tasks)): #(len(training_tasks))
    task={}
    task_file = str(training_path / training_tasks[i])
    
    with open(task_file, 'r') as f:
        task = json.load(f)

    dict_tasks[i] = task
    dict_tasks_shape[i] = {len(task['train']),  len(task['test'])}
    
# Converting into list of tuple 
list_tasks = [v for k, v in dict_tasks.items()] 

In [ ]:
# ======================================================================================
# HERE WE CHANGE THE TASK WE WANT TO WORK WITH
# ======================================================================================

task = list_tasks[11] #images for filling the constraing: 0, 2, 4, 5, 6, 9, 10, 11, 14, ...


In [ ]:
# =========================================================================
# SHAPE OF IMAGES -- input images and output images have usually different shapes. We want to identify them.
# CAREFUL: inside the training images, not all of them have the same shape.
# =========================================================================

# Training images
rows_input_images = {}
columns_input_images = {}
rows_output_images = {}
columns_output_images = {}

for i in range (0,len(task['train'])):
    # Input images
    rows_input_images[i] = len(task['train'][i]['input'])
    columns_input_images[i] = len(task['train'][i]['input'][0])
    
    #Output images
    rows_output_images[i] = len(task['train'][i]['output'])
    columns_output_images[i] = len(task['train'][i]['output'][0])
    
# Convert into LIST
rows_input_images = [ v for k, v in rows_input_images.items()] 
columns_input_images = [ v for k, v in columns_input_images.items()]
rows_output_images = [ v for k, v in rows_output_images.items()] 
columns_output_images = [ v for k, v in columns_output_images.items()]


In [ ]:
# ===========================================================================
# WORKING WITH COLOURS IN THE NEURAL NETWORK
# We divide the values of matrices over 10, to have all the elements of the matrices between [0,1]
# We use the SIGMOID FUNCTION, so the output is between [0,1]
# So here, if the output is between those values, we give them a specific value, so that then we can multiply again by 10 and plot it.
# ==========================================================================
def identify_colors (output_matrix):
    for i in range(0,rows_output_images[0]):
        for j in range(0,columns_output_images[0]):
            if 0.05< output_matrix[i,j] <= 0.15:
                output_matrix[i,j] = 0.1
            if 0.15< output_matrix[i,j] <= 0.25:
                output_matrix[i,j] = 0.2
            if 0.25< output_matrix[i,j] <= 0.35:
                output_matrix[i,j] = 0.3
            if 0.35< output_matrix[i,j] <= 0.45:
                output_matrix[i,j] = 0.4
            if 0.45< output_matrix[i,j] <= 0.55:
                output_matrix[i,j] = 0.5
            if 0.55< output_matrix[i,j] <= 0.65:
                output_matrix[i,j] = 0.6
            if 0.65< output_matrix[i,j] <= 0.75:
                output_matrix[i,j] = 0.7
            if 0.75< output_matrix[i,j] <= 0.85:
                output_matrix[i,j] = 0.8
            if 0.85< output_matrix[i,j] <= 0.95:
                output_matrix[i,j] = 0.9
            if 0.95<output_matrix[i,j]<= 1.0 or 0< output_matrix[i,j]<=0.05:
                output_matrix[i,j] = 0.0
                
    return output_matrix

In [ ]:
# ======================================================================
# WHICH MODEL TO USE? 
# ======================================================================


class modelNeuralNetwork () :
    def __init__(self,rows_input_images, columns_input_images, rows_output_images, columns_output_images ):
        
        # We need this value to specify the shape of the NeuralNetwork of 3 layers. We want to make it proportional as we usually start with less inputs and we finish with more inputs.
        # Example: Input--> (3x3)=9 neurons. Output --> (9x9) = 81 neurons. The hidden layers must have a number of neurons between those.
        neurons_to_add = int((rows_output_images[0]*columns_output_images[0] - rows_input_images[0]*columns_input_images[0])/3) #we have 3 layers in our NN
        
        # We initialise the weights with its sizes
        self._weights1 = np.random.uniform(low=0, high=1, size=(rows_input_images[0]*columns_input_images[0] + neurons_to_add, rows_input_images[0]*columns_input_images[0])) #(33,9)
        self._weights2 = np.random.uniform(low=0, high=1, size=(rows_output_images[0]*columns_output_images[0] - neurons_to_add,rows_input_images[0]*columns_input_images[0] + neurons_to_add))#(57,33)
        self._weights3 = np.random.uniform(low=0, high=1, size=(rows_output_images[0]*columns_output_images[0],rows_output_images[0]*columns_output_images[0] - neurons_to_add)) #(81,57)
        
        self._z1 = {}
        self._z2 = {}
        self._z3 = {}
        
        self._a0 = {}
        self._a1 = {}
        self._a2 = {}
        self._a3 = {}
                
        self._b1 = np.zeros((rows_input_images[0]*columns_input_images[0] + neurons_to_add,1))
        self._b2 = np.zeros((rows_output_images[0]*columns_output_images[0] - neurons_to_add,1))
        self._b3 = np.zeros((rows_output_images[0]*columns_output_images[0],1))
        
        self._error3 = {}
        self._error2 = {}
        self._error1 = {}
        
        self._m = 1 #we work with 1 image at a time
        self._alpha = 0.05

    def forward_propagation(self, X_input_image):
        self._a0 = X_input_image
        
        self._z1 = np.matmul(self._weights1, self._a0) + self._b1
        self._a1 = 1 / (1+np.exp(-self._z1))
        
        self._z2 = np.matmul(self._weights2, self._a1) + self._b2
        self._a2 = 1 / (1+np.exp(-self._z2))
        
        self._z3 = np.matmul(self._weights3, self._a2) + self._b3
        self._a3 = 1 / (1+np.exp(-self._z3))
        return self._a3
        
    
    def backward_propagation(self, y_output_image):
        self._error3 = self._a3 - y_output_image
        
        aux1 = np.matmul(np.transpose(self._weights3), self._error3)
        aux2 = 1 / (1+np.exp(-self._z2)) * (1- 1 / (1+np.exp(-self._z2)))
        self._error2 = np.multiply(aux1, aux2) 
        
        aux1 = np.matmul(np.transpose(self._weights2), self._error2)
        aux2 = 1 / (1+np.exp(-self._z1)) * (1- 1 / (1+np.exp(-self._z1)))
        self._error1 = np.multiply(aux1, aux2) 
        
        
    def update_weights(self):
        self._weights1 = self._weights1 - self._alpha/self._m* np.matmul(self._error1, np.transpose(self._a0))
        self._weights2 = self._weights2 - self._alpha/self._m* np.matmul(self._error2, np.transpose(self._a1))
        self._weights3 = self._weights3 - self._alpha/self._m* np.matmul(self._error3, np.transpose(self._a2))
        
        a = (1,1)
        ones = np.ones(a)
        
        self._b1 = self._b1 - self._alpha/self._m * np.matmul(self._error1, np.transpose(ones))
        self._b2 = self._b2 - self._alpha/self._m * np.matmul(self._error2, np.transpose(ones))
        self._b3 = self._b3 - self._alpha/self._m * np.matmul(self._error3, np.transpose(ones))
        
        return self._weights1, self._weights2, self._weights3
      
        
    def predict(self, X_test_image, weight1avg, weight2_avg, weight3_avg):
        self._a0 = X_test_image
        
        self._z1 = np.matmul(weight1avg, self._a0) + self._b1
        self._a1 = 1 / (1+np.exp(-self._z1))
        
        self._z2 = np.matmul(weight2_avg, self._a1) + self._b2
        self._a2 = 1 / (1+np.exp(-self._z2))
        
        self._z3 = np.matmul(weight3_avg, self._a2) + self._b3
        self._a3 = 1 / (1+np.exp(-self._z3))
        self._a3 = self._a3.reshape((rows_output_images[0],columns_output_images[0]))
        self._a3 = identify_colors(self._a3)
        self._a3 = self._a3 * 10
        print('Predicted Image')
        plt.imshow(self._a3, cmap=cmap, norm=norm)
        plt.show()

        

In [ ]:
# ===============================================================
# INITIALISE THE MODEL
# ===============================================================
model = modelNeuralNetwork(rows_input_images, columns_input_images, rows_output_images, columns_output_images)

In [ ]:
# ==============================================================
# THIS FUNCTION TAKES AN ARRAY AND SEE IF ALL THE ELEMENTS ARE EQUAL
# For example, arr1 = {1,1,1,1,1}-- Output=True
#              arr2 = {1,1,2,1,1}-- Output=False
# ==============================================================

def checkEqualityOfDimensionsOfImages(iterator):
   return len(set(iterator)) <= 1

In [ ]:
# ======================================================================
# TRAIN THE MODEL
# ======================================================================

outputs = {}
weights = {}

# If the input images have same shape. And if the ouput images have same shape
# Input images: all of them (3x3) -->  Output images: all of them (9x9). This works fine!
if (checkEqualityOfDimensionsOfImages(rows_input_images) == True) and (checkEqualityOfDimensionsOfImages(columns_input_images) == True) and (checkEqualityOfDimensionsOfImages(rows_output_images) == True) and (checkEqualityOfDimensionsOfImages(columns_output_images) == True):
    print('These are the images of the training.')
    plot_task(task)
    print('These are the predicted images of the training.')
    for i in range(0, len(task['train'])): #all the examples of the training images of the task
        X_input_image = task['train'][i]['input']

        X_input_image = np.asarray(X_input_image)/10
        X_input_image = X_input_image.reshape((-1, 1)) 
    
        y_output_image = task['train'][i]['output']
        y_output_image = np.asarray(y_output_image)
        y_output_image = y_output_image.reshape((-1, 1))/ 10
    
        for iter in range (0,50): # NUMBER OF TIMES TRAINED
            a3 = model.forward_propagation(X_input_image)

            model.backward_propagation(y_output_image)
            weights1, weights2, weights3 = model.update_weights()
        
        #Store the weights and the outputs
        weights[i] = [weights1, weights2, weights3]
        outputs[i] = a3


        outputs[i] = outputs[i].reshape((rows_output_images[0],columns_output_images[0]))
        outputs[i] = identify_colors(outputs[i])
        outputs[i] = outputs[i] * 10
        
       

        plt.imshow(outputs[i], cmap=cmap, norm=norm)
        plt.show()
else:
    print('The shape of the images of this task are not equal.')

In [ ]:
# =======================================================================
# PREDICTION
# =======================================================================

# Load the test image. 
X_test_image = task['test'][0]['input']


# Predict its output
if (checkEqualityOfDimensionsOfImages(rows_input_images) == True) and (checkEqualityOfDimensionsOfImages(columns_input_images) == True) and (checkEqualityOfDimensionsOfImages(rows_output_images) == True) and (checkEqualityOfDimensionsOfImages(columns_output_images) == True):
    print('Input image of the test data')
    plt.imshow(X_test_image, cmap=cmap, norm=norm)
    plt.show()
    
    #the weights--> i want them to be the average
    weight1_sum = 0
    weight2_sum = 0
    weight3_sum = 0

    for i in range (0, len(task['train'])):
        weight1_sum += (weights[i][0])
        weight2_sum += (weights[i][1])
        weight3_sum += (weights[i][2])
    

    weight1_avg = weight1_sum/len(task['train'])
    weight2_avg = weight2_sum/len(task['train'])
    weight3_avg = weight3_sum/len(task['train'])

    

    X_test_image = np.asarray(X_test_image)/10
    X_test_image = X_test_image.reshape((-1, 1))
    model.predict(X_test_image, weight1_avg, weight2_avg, weight3_avg)


    ## problem: it's NOT learning how the task works. It learns what pixels to colour but with a new example, those weights are not valid.
    # overfitting 
else:
    print('The shape of the images of this task are not equal.')

# The predicted image is not equal :(

# Thanks for your time. Any help is appreciated! We have to solve this! 
